In [1]:
#from textblob import TextBlob
#from attributegetter import *
from generatengrams import ngrammatch
from Contexts import *
import json
from Intents import *
import random
import os
import re
from objectpath import  *


In [2]:

def getEntities(dbPath, filepath, searchQuery):
    
    with open(dbPath, 'r') as f:
        data = json.load(f)

    textfile = open(filepath, 'w')
    
    r = Tree(data).execute(searchQuery)
    for i in r:
       # print(i)
        textfile.write(i)
        textfile.write("\n")
    
    textfile.close()
    
    return

def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent
    as mentioned in the intent config file'''
    '''Performs actions pertaining to current intent
    for action in current_intent.actions:
        if action.contexts_satisfied(active_contexts):
            return perform_action()
    '''

    context = IntentComplete()
    return 'action: ' + current_intent.action, context


def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    
    for para in current_intent.params:
        if para.required:
            if para.name not in attributes:
                #Example of where the context is born, implemented in Contexts.py
                
                if para.name=='title':
                    context = GetTitle() 
                    
                if para.name=='author':
                    context = GetAuthor()
                
                if para.name=='subject':
                    context = GetSubject() 
                
                if para.name=='cuisine':
                    context = GetCuisine() 
                
                if para.name=='location':
                    context = GetLocation() 
                
                if para.name=='cost':
                    context = GetCost() 
                    
                if para.name=='RegNo':
                    context = GetRegNo()
                    
                #returning a random prompt frmo available choices.
                return random.choice(para.prompts), context

    return None, context


def input_processor(user_input, context, attributes, intent):
    '''Spellcheck and entity extraction functions go here'''
    
    #uinput = TextBlob(user_input).correct().string
    
    #update the attributes, abstract over the entities in user input
    attributes, cleaned_input = getattributes(user_input, context, attributes)
    
    return attributes, cleaned_input

def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        print('Intent loaded' + intent['intentname'])
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    print(clean_input)
    #Scoring Algorithm, can be changed.
    scores = ngrammatch(clean_input)
    
    #choosing here the intent with the highest score
    scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])

    print('scores', scores)
    

    if(current_intent==None):
        if(clean_input=="pick a book"):
            return loadIntent('params/newparams.cfg', 'PickBook')
        if(clean_input=="find restaurant"):
            return loadIntent('params/newparams.cfg','SearchRestaurant')
        else:
            return loadIntent('params/newparams.cfg',scores[-1][0])
        
    else:
        #If current intent is not none, stick with the ongoing intent
        print('Intent loaded' + current_intent.name)
        return current_intent

def getattributes(uinput,context,attributes):
    '''This function marks the entities in user input, and updates
    the attributes dictionary'''
    #Can use context to context specific attribute fetching
    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:
        #Code can be optimised here, loading the same files each time suboptimal 
        if context.name=='FirstGreeting':
            files = os.listdir('./entities/')
            entities = {}
            for fil in files:
                lines = open('./entities/'+fil).readlines()
                for i, line in enumerate(lines):
                    lines[i] = line[:-1]
            
                entities[fil[:-4]] = '|'.join(lines)
            
            #Extract entity and update it in attributes dict
            for entity in entities:
                for i in entities[entity].split('|'):
                    if i.lower() in uinput.lower():
                        attributes[entity] = i
            for entity in entities:
                    uinput = re.sub(entities[entity],r'$'+entity,uinput,flags=re.IGNORECASE)

        #Example of where the context is being used to do conditional branching.
       
        if context.name=='GetTitle' and context.active:
                attributes['title'] = uinput
                context.active=False
        
        if context.name=='GetAuthor' and context.active:
                attributes['author'] = uinput
                context.active=False
        
        if context.name=='GetSubject' and context.active:
                attributes['subject'] = uinput
                context.active=False
        
        if context.name=='GetCuisine' and context.active:
                attributes['cuisine'] = uinput
                context.active=False
        
        if context.name=='GetLocation' and context.active:
                attributes['location'] = uinput
                context.active=False
        
        if context.name=='GetCost' and context.active:
                attributes['cost'] = uinput
                context.active=False
        
        if context.name=='GetRegNo' and context.active:
                attributes['RegNo'] = uinput
                context.active=False
            
        return attributes, uinput

In [ ]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):
        
        '''Initialise a default session'''
        
        #Active contexts not used yet, can use it to have multiple contexts
        self.active_contexts = active_contexts
        
        #Contexts are flags which control dialogue flow, see Contexts.py        
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        #self.current_intent = First_Greeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()

    def reply(self, user_input):
        '''Generate response to user input'''
        
        getEntities("./BooksData.json", "./entities/title.dat", "$.books.title")
        getEntities("./BooksData.json", "./entities/subject.dat", "$.books.subject")
        getEntities("./BooksData.json", "./entities/author.dat", "$.books.author")
        
        getEntities("./restaurantsData.json", "./entities/cuisine.dat", "$.restaurants.cuisine")
        getEntities("./restaurantsData.json", "./entities/location.dat", "$.restaurants.location")
        getEntities("./restaurantsData.json", "./entities/cost.dat", "$.restaurants.cost")
        
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        
        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)

        #prompt being None means all parameters satisfied, perform the intent action  
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)
             
            if self.context.name =='IntentComplete':
                print(self.attributes)
            
             #db code - intent action
             #load data
            
        
             #formulate query
            #a ="'Unknown'"
            #b = "'The Epic Of Gilgamesh'"
            #c = "'Akkadian'"
            
            print("the current intent is" + self.current_intent.name)
            if self.current_intent.name =='PickBook':
                with open('./BooksData.json', 'r') as f:
                     data = json.load(f)
                a = "'" + self.attributes['author'] + "'"
                b = "'" + self.attributes['title'] + "'"
                c = "'" + self.attributes['subject'] + "'"
                #str = "$.books[@.author is " + "'"+ author + "'" + "and @.title is " + "'" + title + "' and " + "@.subject is " + "'" + subject + "'].title"
                #str= "$.books.title"
                #str = "$.books[@.author is " + a + "and @.title is " + b + " and " + "@.subject is " + c + "]"
                str = "$.books["+a+" in @.author  or  "+b+" in @.title or "+c+" in @.subject]"
                print(str)
                #run query
                try:
                    resultset = Tree(data).execute(str)
                    for i in resultset:
                            print (i) 
                except:
                    print("No Match, Please try again!")
                
        
            if self.current_intent.name =='SearchRestaurant':
                with open('./restaurantsData.json', 'r') as f:
                     data = json.load(f)
                a = "'" + self.attributes['cuisine'] + "'"
                b = "'" + self.attributes['location'] + "'"
                c = "'" + self.attributes['cost'] + "'"  
                str = "$.restaurants["+a+" in @.cuisine  and  "+b+" in @.location and "+c+" in @.cost]"
                #str = "$.restaurants.location"
                #run query
                print(str)
                try:
                    resultset = Tree(data).execute(str)
                    for i in resultset:
                            print (i) 
                except:
                    print("No Match, Please try again!")
                 
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
        
        return prompt

In [ ]:
session = Session()

print ('BOT: Hi! How may I assist you?')

while True:
    
    inp = input('User: ')
    print ('BOT:', session.reply(inp))

BOT: Hi! How may I assist you?
User: find restaurant
find restaurant
scores [('PickBook', 0.06690705128205128), ('SearchRestaurant', 0.32064777327935223)]
Intent loadedSearchRestaurant
in Cuisine context
BOT: For which cuisine are you looking for ?
User: indian
indian
scores [('PickBook', 0.06690705128205128), ('SearchRestaurant', 0.08477732793522268)]
Intent loadedSearchRestaurant
in Location context
BOT: Please mention the location name such as east,west,north,south.
User: south
south
scores [('PickBook', 0.06690705128205128), ('SearchRestaurant', 0.08477732793522268)]
Intent loadedSearchRestaurant
in Cost context
BOT: What cost are you looking for?
User: medium
medium
scores [('PickBook', 0.06690705128205128), ('SearchRestaurant', 0.08477732793522268)]
Intent loadedSearchRestaurant
{'location': 'south', 'cost': 'medium', 'cuisine': 'indian'}
the current intent isSearchRestaurant
$.restaurants['indian' in @.cuisine  and  'south' in @.location and 'medium' in @.cost]
BOT: action: Rest